- **Title: Tutorial of LocalCat**
- **Author**: Ewen Wang (ewang1@volvocars.com)
- **Last Update**: March 4, 2024

This tutorial will guide you through the process of using **LocalCat** to load pre-trained LLMs and fine-tune them with domain data.

# Load Pre-trained LLMs

**LocalCat** simplifies the process of loading pre-trained LLMs.

## Load Packages

In [ ]:
# pip install -r requirements.txt

In [ ]:
import pandas as pd

from LocalCat.Translate import Translate
from LocalCat.Translate import Local

In [ ]:
model_mbart = "facebook/mbart-large-50-many-to-many-mmt"

trans = Translate(model_name_or_path=model_mbart,
                  src_lang='zh_CN',
                  tgt_lang='en_XX')

**Note:**

For AI engineers in China, you may use hugging face mirror sites, such as [hf-mirror.com](https://hf-mirror.com/). 
    
``` bash
HF_ENDPOINT=https://hf-mirror.com python3 download_model.py
```

## Translate by Case

In [ ]:
text = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"
print(trans.translator(text=text))

## Translate on Batch

**LocalCat** also supports batch translation. All you need to do is to load the data and call the `translator_batch` method.

The data should be in the form of a `pandas` DataFrame.

In [ ]:
file_inference = "../data/trans/INFERENCE_SAMPLE.csv"

df = pd.read_csv(file_inference)
df.head()

In [ ]:
df = trans.translator_batch(df=df, 
                            col_src='Chinese', 
                            col_tgt="English")

In [ ]:
df.head()

# Fine-tune LLMs with Domain Data

**LocalCat** also supports fine-tuning pre-trained LLMs with domain data.

![](../images/finetune.png)

## Fine-tune LLM

To fine-tune a LLM, you need some labelled data. Here we simplify it as a `pandas` dataframe. 

For translation task, spicificly, you need the source text and target text to train the model.

In [ ]:
file_training = "../data/trans/PROC-NCVQS-2023.csv"

df = pd.read_csv(file_training)

In [ ]:
df.head()

In [ ]:
# model_mbart = "facebook/mbart-large-50-many-to-many-mmt"
# trans = Translate(model_mbart)

finetuned_model_path = "../models/mbart-finetuned-cn-to-en-auto-2023"

trans.finetune(df=df, 
               finetuned_model_path=finetuned_model_path,
               train_size=0.98, 
               batch_size=4,
               learning_rate=2e-5,
               num_train_epochs=1)

In [ ]:
finetuned_model_path = "../models/mbart-finetuned-cn-to-en-auto-2023"

text = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"

trans = Translate(model_name_or_path=finetuned_model_path)
print(trans.translator(text=text))

# Deploy the LLM

**LocalCat** also supports deploying LLMs on the cloud (AWS Sagemaker Endpoint). 

![](../images/aws_llm.png)

Deploying the model contains the following steps:

1. Push the model to S3
2. Deploy the model as an endpoint
3. Test the endpoint

In [ ]:
model_path = "../models/"
model_finetuned = "mbart-finetuned-cn-to-en-auto-2023"

## Step 1: Push the model to S3

In [ ]:
bucket = "ai"
prefix = "llm"

local = Local(model_name=model_finetuned, model_path=model_path)
local.push_to_s3(bucket=bucket, prefix=prefix)

## Step2: Deploy the model as an endpoint

In [ ]:
local.deploy(instance_type='ml.g4dn.4xlarge',
             transformers_version='4.37.0', 
             pytorch_version='2.1.0', 
             py_version='py310')

## Step 3: Test the endpoint

Check the endpoint name in the AWS Sagemaker Console, say `MBART-20240226-024324`.

In [ ]:
local = Local()
local.endpoint_name = "MBART-20240226-024324" 

text = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"
result = local.translator(text=text)
print(result)